# クライアント設定

In [ ]:
import msal
import requests

TENANT_ID = "REPLACE_YOUR_TENANT_ID"
CLIENT_ID = "REPLACE_YOUR_CLIENT_ID"

# Scopes must match the delegated permissions you added in Entra ID
SCOPES = [
    "Calendars.Read",
    "Chat.Read",
    "OnlineMeetings.Read",
    "OnlineMeetingTranscript.Read.All",
    "Team.ReadBasic.All",
    "Channel.ReadBasic.All",
    "ChannelMessage.Read.All",
    "User.Read",
]

AUTHORITY = f"https://login.microsoftonline.com/{TENANT_ID}"

app = msal.PublicClientApplication(
    client_id=CLIENT_ID,
    authority=AUTHORITY,)

flow = app.initiate_device_flow(scopes=SCOPES)
if "user_code" not in flow:
    raise RuntimeError(f"Failed to create device flow: {flow}")

print(flow["message"])  # User instructions

result = app.acquire_token_by_device_flow(flow)

if "access_token" not in result:
    raise RuntimeError(f"Failed to acquire token: {result}")

access_token = result["access_token"]

headers = {"Authorization": f"Bearer {access_token}"}

# Example 1: List chats the user is in
r = requests.get("https://graph.microsoft.com/v1.0/me/chats", headers=headers)
print("me/chats:", r.status_code)
print(r.text)

# Example 2: Joined teams
r = requests.get("https://graph.microsoft.com/v1.0/me/joinedTeams", headers=headers)
print("me/joinedTeams:", r.status_code)
print(r.text)


# Teams チャネル/グループチャット/会議データの取得

このセクションでは、すでに取得済みの `access_token` / `headers` を使って、次の情報を取得するコードを追加します。



- 特定チーム配下のチャネル一覧とチャネル ID 取得

- グループチャット & 会議チャットの ID 取得

- 特定チャネルの全メッセージ（＋必要であればスレッド返信）

- 特定グループチャットの全メッセージ

- ユーザーが主催/参加しているオンライン会議のメタデータ & 参加者

- 会議トランスクリプト一覧 & コンテンツ取得（権限がある場合）

- 会議に紐づくチャット（chatType = `meeting`）のメッセージ取得



※ 実際に API を呼び出すには、Microsoft Graph 側で必要な権限が付与されている必要があります。権限不足の場合は 403 などのエラーになります。

In [ ]:
import json

from typing import List, Dict, Any



# 共通: @odata.nextLink をたどって全件取得するヘルパー

def get_with_paging(url: str, headers: Dict[str, str], params: Dict[str, Any] | None = None) -> List[Dict[str, Any]]:

    items: List[Dict[str, Any]] = []

    while url:

        print(f"GET {url}")

        resp = requests.get(url, headers=headers, params=params)

        print("  ->", resp.status_code)
        print(resp.text)

        resp.raise_for_status()

        data = resp.json()

        items.extend(data.get("value", []))

        # 次ページがある場合は @odata.nextLink に URL が入る

        url = data.get("@odata.nextLink")

        # 2 ページ目以降は params は不要

        params = None

    return items



# 1. チーム一覧 & 各チーム配下のチャネル一覧を取得して ID を確認

teams_url = "https://graph.microsoft.com/v1.0/me/joinedTeams"

teams = get_with_paging(teams_url, headers)



print("==== Joined Teams ====")

for t in teams:

    print(f"Team: {t.get('displayName')} | id={t.get('id')}")



print("\n==== Channels per Team ====")

for t in teams:

    team_id = t.get("id")

    channels_url = f"https://graph.microsoft.com/v1.0/teams/{team_id}/channels"

    channels = get_with_paging(channels_url, headers)

    print(f"\nTeam: {t.get('displayName')} ({team_id})")

    for ch in channels:

        print(f"  Channel: {ch.get('displayName')} | id={ch.get('id')}")



# 2. グループチャット & 会議チャットの ID 一覧

#   chatType: 'oneOnOne' | 'group' | 'meeting'

group_chats_url = "https://graph.microsoft.com/v1.0/me/chats?$filter=chatType eq 'group'"

group_chats = get_with_paging(group_chats_url, headers)



print("\n==== Group Chats (chatType = group) ====")

for chat in group_chats:

    print(f"Group Chat: {chat.get('topic')} | id={chat.get('id')} | lastMessagePreview={chat.get('lastMessagePreview', {}).get('body', {}).get('content')}")



meeting_chats_url = "https://graph.microsoft.com/v1.0/me/chats?$filter=chatType eq 'meeting'"

meeting_chats = get_with_paging(meeting_chats_url, headers)



print("\n==== Meeting Chats (chatType = meeting) ====")

for chat in meeting_chats:

    # onlineMeetingInfo や topic からどの会議かを識別できます

    print("Meeting Chat:")

    print("  id=", chat.get("id"))

    print("  topic=", chat.get("topic"))

    print("  onlineMeetingInfo=", json.dumps(chat.get("onlineMeetingInfo"), ensure_ascii=False))

    print()

In [ ]:
# ID 設定セル（下記はサンプル値です。自分の環境の値に置き換えてください）
# 上の Teams / グループチャット / Meeting チャット一覧セルの出力を見ながら、
# 利用したい対象の ID や join URL をここにセットしてください。

# チーム チャネル用
team_id = "REPLACE_TEAM_ID"       # 例: 上の Joined Teams の出力から対象チームの id
channel_id = "REPLACE_CHANNEL_ID"    # 例: 上の Channels per Team の出力から対象チャネルの id

# グループチャット用
group_chat_id = "REPLACE_GROUP_CHAT_ID"  # 例: Group Chats の出力から対象チャットの id

# 会議チャット / オンライン会議用
meeting_chat_id = "REPLACE_MEETING_CHAT_ID"  # 例: Meeting Chats の出力から対象チャットの id
join_url = "REPLACE_MEETING_JOIN_URL"         # 例: Meeting チャットの onlineMeetingInfo.joinWebUrl など

## 特定チャネルの全メッセージ取得

上のセルで確認した `team_id` と `channel_id` を使って、特定チャネルの全メッセージ（＋必要に応じて返信）を取得します。



- メッセージ本体: `/teams/{team-id}/channels/{channel-id}/messages`

- 各メッセージの返信: `/teams/{team-id}/channels/{channel-id}/messages/{message-id}/replies`



取得したメッセージは Python のリストとして返すので、後続で pandas に流し込んだり、ファイル保存したりできます。

In [ ]:
from datetime import datetime

from pathlib import Path
import json



# メッセージ共通: 取得したメッセージを output 配下に JSON で保存するヘルパー

OUTPUT_DIR = Path("output")

def save_messages_to_json(messages: List[Dict[str, Any]], prefix: str) -> str:
    """取得したメッセージ一覧を output 配下に JSON で保存"""
    OUTPUT_DIR.mkdir(exist_ok=True)
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_path = OUTPUT_DIR / f"{prefix}_{ts}.json"
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(messages, f, ensure_ascii=False, indent=2)
    print(f"Saved {len(messages)} messages to {out_path}")
    return str(out_path)



def get_channel_messages(team_id: str, channel_id: str) -> List[Dict[str, Any]]:

    """特定チャネルの全メッセージを取得"""

    url = f"https://graph.microsoft.com/v1.0/teams/{team_id}/channels/{channel_id}/messages"

    return get_with_paging(url, headers)



def get_channel_message_replies(team_id: str, channel_id: str, message_id: str) -> List[Dict[str, Any]]:

    """特定メッセージのスレッド返信を全件取得"""

    url = f"https://graph.microsoft.com/v1.0/teams/{team_id}/channels/{channel_id}/messages/{message_id}/replies"

    return get_with_paging(url, headers)



In [ ]:
# 使い方例: 事前に ID 設定セルで team_id / channel_id をセットしておく

messages = get_channel_messages(team_id, channel_id)

print(len(messages), "messages")

# 先頭 3 件をタイトル＋送信者＋送信日時だけ表示

for m in messages:

    from_user = (m.get("from", {}) or {}).get("user", {}) or {}

    user_name = from_user.get("displayName")

    created = m.get("createdDateTime")

    print(created, "|", user_name, "|", (m.get("body", {}) or {}).get("content"))


"""取得したチャネルメッセージを output 配下に JSON で保存"""
save_messages_to_json(messages, "channel_messages")

## 特定グループチャットの全メッセージ取得

グループチャット ID（上の `Group Chats` の一覧で表示される `id`）を指定して、そのチャットの全メッセージを取得します。



- メッセージ本体: `/chats/{chat-id}/messages`

- 1 対 1 チャットでもグループチャットでも同じエンドポイントで取得できます。



ここでは主に `chatType = group` のチャットを想定しています。

In [ ]:
def get_chat_messages(chat_id: str) -> List[Dict[str, Any]]:

    """特定チャット (1:1 / グループ / 会議チャット) の全メッセージを取得"""

    url = f"https://graph.microsoft.com/v1.0/chats/{chat_id}/messages"

    return get_with_paging(url, headers)

In [ ]:
# 使い方例: 事前に ID 設定セルで group_chat_id をセットしておく

chat_messages = get_chat_messages(group_chat_id)

print(len(chat_messages), "messages in group chat")

for m in chat_messages:

    from_user = (m.get("from", {}) or {}).get("user", {}) or {}

    user_name = from_user.get("displayName")

    created = m.get("createdDateTime")

    print(created, "|", user_name, "|", (m.get("body", {}) or {}).get("content"))


"""取得したグループチャットのメッセージを output 配下に JSON で保存"""
save_messages_to_json(chat_messages, "group_chat_messages")

## オンライン会議のメタデータ・参加者・トランスクリプト・会議チャット

オンライン会議まわりの情報取得は次のように行います。



1. `/me/onlineMeetings` でユーザーが主催/参加するオンライン会議一覧を取得

    - `id`, `subject`, `startDateTime`, `endDateTime`, `joinWebUrl` などが含まれます

    - `participants` の中に参加者情報があります

2. 特定会議に対して `/me/onlineMeetings/{meeting-id}/transcripts` でトランスクリプト一覧

    - さらに `/me/onlineMeetings/{meeting-id}/transcripts/{transcript-id}/content` で実際のテキスト (通常は VTT) を取得

    - これらの API はテナント側でトランスクリプト機能が有効 & 必要な Graph 権限が付与されている必要があります

3. 会議中チャットは、`chatType = 'meeting'` のチャットとして `/me/chats` から取得可能

    - 上の「Meeting Chats」一覧の `id` を `get_chat_messages` に渡せば、その会議チャットの全メッセージを取得できます



> 備考: トランスクリプト関連 API は組織全体/app 権限や RSC (resource-specific consent) 権限を前提としていることが多いため、

> 単純なユーザー委任トークンだけだと 403 になる場合があります。その場合はテナント管理者に Graph 権限設定を相談してください。

In [ ]:
from urllib.parse import quote

from datetime import datetime



def get_online_meeting_by_join_url(join_url: str) -> Dict[str, Any] | None:

    """joinWebUrl をキーに単一の onlineMeeting を取得



    /me/onlineMeetings は『一覧』ではなくフィルタ取得が基本で、

    例: GET /me/onlineMeetings?$filter=JoinWebUrl eq '{joinWebUrl}'



    join_url は Meeting チャットの onlineMeetingInfo や、

    カレンダー イベントの onlineMeeting プロパティから取得できます。

    """



    url = "https://graph.microsoft.com/v1.0/me/onlineMeetings"

    # requests に任せて $filter をクエリとして渡す

    flt = f"JoinWebUrl eq '{join_url}'"

    params = {"$filter": flt}



    print("GET", url, "? $filter=", flt)

    resp = requests.get(url, headers=headers, params=params)

    print("  ->", resp.status_code)

    print(resp.text)

    resp.raise_for_status()



    data = resp.json()

    meetings = data.get("value", []) or []

    if not meetings:

        print("No onlineMeeting found for this joinWebUrl")

        return None

    if len(meetings) > 1:

        print("Warning: multiple onlineMeetings returned. Using the first one.")

    return meetings[0]



def show_meeting_participants(meeting: Dict[str, Any]) -> None:

    """onlineMeeting オブジェクトから参加者情報をざっくり表示"""

    participants = meeting.get("participants", {}) or {}

    organizer = participants.get("organizer", {}) or {}

    attendees = participants.get("attendees", []) or []



    print("Organizer:")

    print("  ", (organizer.get("upn") or organizer.get("identity", {}).get("user", {}).get("displayName")))



    print("Attendees:")

    for a in attendees:

        identity = a.get("upn") or (a.get("identity", {}).get("user", {}) or {}).get("displayName")

        role = a.get("role")

        print("  ", identity, "| role=", role)



def list_meeting_transcripts(meeting_id: str) -> List[Dict[str, Any]]:

    """特定オンライン会議のトランスクリプト一覧を取得"""

    url = f"https://graph.microsoft.com/v1.0/me/onlineMeetings/{meeting_id}/transcripts"

    resp = requests.get(url, headers=headers)

    print("GET", url, "->", resp.status_code)

    print(resp.text)

    resp.raise_for_status()

    data = resp.json()

    transcripts = data.get("value", [])

    for t in transcripts:

        print("Transcript id=", t.get("id"), "| created=", t.get("createdDateTime"))

    return transcripts



def download_meeting_transcript_content_vtt(meeting_id: str, transcript_id: str, out_path: str) -> None:
    """
    onlineMeetings の transcript を VTT でダウンロードして保存
    """
    url = f"https://graph.microsoft.com/v1.0/me/onlineMeetings/{meeting_id}/transcripts/{transcript_id}/content"
    params = {"$format": "text/vtt"}

    hdrs = dict(headers)
    hdrs["Accept"] = "text/vtt"

    resp = requests.get(url, headers=hdrs, params=params)
    print("GET", resp.url, "->", resp.status_code)
    print(resp.text[:500])  # 失敗時の原因確認用（成功時はVTTの先頭が出る）

    resp.raise_for_status()

    # content はバイナリで返ることがあるので wb が安全
    with open(out_path, "wb") as f:
        f.write(resp.content)

    print("Saved transcript content to", out_path)



def download_meeting_transcript_content(meeting_id: str, transcript_id: str, out_path: str) -> None:

    """単一トランスクリプトの content を取得してファイル保存 (VTT など)

    NOTE: 一部テナント/エンドポイントでは /me パスで /content にアクセスすると 400 になるため、

          実運用では callTranscripts API (download_meeting_transcript_content_via_call_transcript) の利用を推奨します。"""

    url = f"https://graph.microsoft.com/v1.0/me/onlineMeetings/{meeting_id}/transcripts/{transcript_id}/content"

    resp = requests.get(url, headers=headers)

    print("GET", url, "->", resp.status_code)

    print(resp.text[:500])

    resp.raise_for_status()

    with open(out_path, "w", encoding="utf-8") as f:

        f.write(resp.text)

    print("Saved transcript content to", out_path)



def download_meeting_transcript_content_by_url(transcript_content_url: str, out_path: str) -> None:

    """transcriptContentUrl を使ってトランスクリプトの content を取得してファイル保存

    ※ このエンドポイントも環境によっては 400 になる場合があります。"""

    resp = requests.get(transcript_content_url, headers=headers)

    print("GET", transcript_content_url, "->", resp.status_code)

    print(resp.text[:500])

    resp.raise_for_status()

    with open(out_path, "w", encoding="utf-8") as f:

        f.write(resp.text)

    print("Saved transcript content to", out_path)

In [ ]:
# 使い方例: 事前に ID 設定セルで join_url / meeting_chat_id をセットしておく

# 1. meeting_chats 一覧セルを実行して、対象会議の onlineMeetingInfo を確認

# 2. その中の joinWebUrl を ID 設定セルの join_url にペースト



meeting = get_online_meeting_by_join_url(join_url)

if meeting:

    show_meeting_participants(meeting)

    m_id = meeting.get("id")

    transcripts = list_meeting_transcripts(m_id)

    if transcripts:

        # transcripts[0]["id"] は callTranscript の ID として使える

        t0_id = transcripts[0].get("id")

        if t0_id:

            download_meeting_transcript_content_vtt(m_id, t0_id, "output/meeting_transcript_0.vtt")

        else:

            print("transcript id がレスポンスに含まれていません")



# 会議中チャットのメッセージは、上の meeting_chats の出力から対象チャットの id を選び、
meeting_chat_messages = get_chat_messages(meeting_chat_id)

print(len(meeting_chat_messages), "messages in meeting chat")

for m in meeting_chat_messages:

    from_user = (m.get("from", {}) or {}).get("user", {}) or {}

    user_name = from_user.get("displayName")

    created = m.get("createdDateTime")

    print(created, "|", user_name, "|", (m.get("body", {}) or {}).get("content"))


"""取得した会議チャットのメッセージを output 配下に JSON で保存"""
save_messages_to_json(meeting_chat_messages, "meeting_chat_messages")